In [2]:
def gini(n1, n2):
    p1 = n1/(n1+n2)
    return 2 * p1 * (1-p1)

In [3]:
gini(486, 214)

0.4245061224489796

In [4]:
import pandas as pd
from sklearn import * 
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline

In [5]:
df = pd.read_csv("/data/credit-default.csv")
df.head()

,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_length,installment_rate,personal_status,other_debtors,...,property,age,installment_plan,housing,existing_credits,default,dependents,telephone,foreign_worker,job
0,< 0 DM,6,critical,radio/tv,1169,unknown,> 7 yrs,4,single male,none,...,real estate,67,none,own,2,1,1,yes,yes,skilled employee
1,1 - 200 DM,48,repaid,radio/tv,5951,< 100 DM,1 - 4 yrs,2,female,none,...,real estate,22,none,own,1,2,1,none,yes,skilled employee
2,unknown,12,critical,education,2096,< 100 DM,4 - 7 yrs,2,single male,none,...,real estate,49,none,own,1,1,2,none,yes,unskilled resident
3,< 0 DM,42,repaid,furniture,7882,< 100 DM,4 - 7 yrs,2,single male,guarantor,...,building society savings,45,none,for free,1,1,2,none,yes,skilled employee
4,< 0 DM,24,delayed,car (new),4870,< 100 DM,1 - 4 yrs,3,single male,none,...,unknown/none,53,none,for free,2,2,2,none,yes,skilled employee


In [24]:
target = "default"
label_encoder = preprocessing.LabelEncoder()

y = label_encoder.fit_transform(df[target])
X = df.drop(columns=target)

X = pd.get_dummies(X, drop_first=True)


X_train, X_test, y_train, y_test = model_selection.train_test_split(X.values * 1.0, y
                                        , test_size = 0.3, random_state = 1)

pipe = pipeline.Pipeline([
    #("poly", preprocessing.PolynomialFeatures(degree=2, include_bias=False)),
    #("scaler", preprocessing.StandardScaler()),
    ("est", tree.DecisionTreeClassifier(random_state = 241, min_impurity_decrease = 0.001, min_samples_leaf = 25
                                        , max_depth = 3))
])

pipe.fit(X_train, y_train)

est = pipe.steps[-1][-1]

y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)

y_test_prob = pipe.predict_proba(X_test)[:, 1]
y_test_pred = np.where(y_test_prob>0.8, 1, 0)

print("Confusion matrix:\n", metrics.confusion_matrix(y_test, y_test_pred))
print("Test accuracy:", metrics.accuracy_score(y_test, y_test_pred))
print("Training accuracy:", metrics.accuracy_score(y_train, y_train_pred))
print("Test recall:", metrics.recall_score(y_test, y_test_pred))
print("Training recall:", metrics.recall_score(y_train, y_train_pred))
print("Test precision:", metrics.precision_score(y_test, y_test_pred))
print("Training precision:", metrics.precision_score(y_train, y_train_pred))
print("Test f1:", metrics.f1_score(y_test, y_test_pred))
print("Training f1:", metrics.f1_score(y_train, y_train_pred))


Confusion matrix:
 [[214   0]
 [ 86   0]]
Test accuracy: 0.7133333333333334
Training accuracy: 0.7385714285714285
Test recall: 0.0
Training recall: 0.35514018691588783
Test precision: 0.0
Training precision: 0.628099173553719
Test f1: 0.0
Training f1: 0.4537313432835821


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [25]:
from sklearn.tree import export_graphviz
export_graphviz(est, out_file = "tree.dot", feature_names = X.columns, filled=True)
!dot -Tpng tree.dot -o tree.png

In [20]:
X.shape

(1000, 48)

In [21]:
(.44 * 239 + .496*185)/424

0.46443396226415096

In [22]:
161/239

0.6736401673640168

In [23]:
486/700

0.6942857142857143

In [28]:
summary = pd.DataFrame(dict(importance = est.feature_importances_, feature = X.columns))
summary.sort_values("importance", ascending = False)

,importance,feature
9,0.558037,checking_balance_unknown
0,0.192398,months_loan_duration
24,0.090326,savings_balance_< 100 DM
4,0.069277,age
39,0.063425,installment_plan_none
1,0.026537,amount
36,0.000000,property_other
28,0.000000,employment_length_4 - 7 yrs
29,0.000000,employment_length_> 7 yrs
30,0.000000,employment_length_unemployed
